In [1]:
import scanpy as sc
import tangram as tg
# https://github.com/broadinstitute/Tangram

In [2]:
ad_sp = sc.read_h5ad("/data/kjc2/projects/P330.CSA/processed_xenium/processed_xenium_CRC_BJM1_S1R1_processed.h5ad")
ad_sc = sc.read_h5ad("/data/kjc2/projects/P330.CSA/processed_xenium/CRC_BJM1_scVIed.h5ad")


# run basic

The function pp_adatas finds the common genes between adata_sc, adata_sp, and saves them in two adatas.uns for mapping and analysis later. Also, it subsets the intersected genes to a set of training genes passed by genes. If genes=None, Tangram maps using all genes shared by the two datasets. Once the datasets are pre-processed we can map:

In [3]:
tg.pp_adatas(ad_sc, ad_sp, genes=None)

# INFO:root:725 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
# INFO:root:725 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
# INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
# INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.

INFO:root:725 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:725 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


The returned AnnData,ad_map, is a cell-by-voxel structure where ad_map.X[i, j] gives the probability for cell i to be in voxel j. This structure can be used to project gene expression from the single cell data to space, which is achieved via tg.project_genes.

In [ ]:
ad_map = tg.map_cells_to_space(ad_sc, ad_sp)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 725 genes and rna_count_based density_prior in cells mode...


The returned ad_ge is a voxel-by-gene AnnData, similar to spatial data ad_sp, but where gene expression has been projected from the single cells. This allows to extend gene throughput, or correct for dropouts, if the single cells have higher quality (or more genes) than spatial data. It can also be used to transfer cell types onto space.

In [ ]:
ad_ge = tg.project_genes(ad_map, ad_sc)

# run cluster level

In [ ]:
ad_map = tg.map_cells_to_space(
               ad_sc, 
               ad_sp,         
               mode='clusters',
               cluster_label='subclass_label')

In [ ]:
ad_ge = tg.project_genes(
              ad_map, 
              ad_sc,
              cluster_label='subclass_label')

In [30]:
tg.pp_adatas(sc_data, st_data, genes=None)
# The function pp_adatas finds the common genes between adata_sc, adata_sp, and saves them in two adatas.uns for mapping and analysis later.
# Also, it subsets the intersected genes to a set of training genes passed by genes.
# If genes=None, Tangram maps using all genes shared by the two datasets.
# Once the datasets are pre-processed we can map:     ad_map = tg.map_cells_to_space(ad_sc, ad_sp)

# INFO:root:4951 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
# INFO:root:4951 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
# INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
# INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.

INFO:root:4951 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:4951 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [ ]:
ad_map = tg.map_cells_to_space(sc_data, st_data)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 4951 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: -0.010
Score: -0.047


In [ ]:
ad_ge = tg.project_genes(ad_map, sc_data)

# performance evaluation

In [ ]:
import tangram as tg
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 메시지 숨기기 (선택 사항)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning) # NaN 관련 경고 숨기기

# --- 1. 데이터 로드 (예시 - 실제 데이터 로드 코드로 대체 필요) ---
# ad_sc = ad.read_h5ad("path/to/your/single_cell.h5ad")
# ad_sp = ad.read_h5ad("path/to/your/spatial_xenium.h5ad")
# print("Single-cell data:", ad_sc)
# print("Spatial data:", ad_sp)

# --- 가상의 AnnData 객체 생성 (실제 데이터 사용 시 이 부분 삭제) ---
# 예시를 위해 작은 랜덤 데이터 생성
# rng = np.random.default_rng(0)
# n_sc_cells, n_genes = 100, 50
# n_sp_cells, n_spots = 200, 30

# # Single-cell data
# ad_sc_X = rng.poisson(1, size=(n_sc_cells, n_genes)).astype(np.float32)
# ad_sc = ad.AnnData(X=ad_sc_X)
# ad_sc.var_names = [f"Gene_{i}" for i in range(n_genes)]
# ad_sc.obs_names = [f"SC_Cell_{i}" for i in range(n_sc_cells)]
# # 공간 데이터에 필요한 마커 유전자 등 추가 정보가 있다면 전처리 단계에서 추가

# # Spatial data (Xenium-like: cell-level)
# ad_sp_X = rng.poisson(1, size=(n_sp_cells, n_genes)).astype(np.float32)
# ad_sp = ad.AnnData(X=ad_sp_X)
# ad_sp.var_names = [f"Gene_{i}" for i in range(n_genes)]
# ad_sp.obs_names = [f"Sp_Cell_{i}" for i in range(n_sp_cells)]
# # Xenium 데이터는 .obsm['spatial']에 세포 좌표 포함
# ad_sp.obsm['spatial'] = rng.random((n_sp_cells, 2)) * 100

# print("Generated dummy single-cell data:", ad_sc)
# print("Generated dummy spatial data (cell-level):", ad_sp)
# print("-" * 30)

# --- 2. 평가 대상 유전자 선정 ---
# 예시: 모든 유전자 사용 또는 특정 목록 정의
# evaluation_genes = ad_sp.var_names.tolist()
# 또는, 예를 들어 상위 10개 유전자만 선택
evaluation_genes = ad_sp.var_names[:10].tolist()
print(f"Selected {len(evaluation_genes)} genes for evaluation: {evaluation_genes}")
print("-" * 30)

# --- 3. Tangram 실행 ---
# Tangram은 공통 유전자를 사용하므로, 미리 필터링하거나 pp_adatas가 처리하도록 둠
# tg.pp_adatas는 내부적으로 공통 유전자를 찾음
# 만약 특정 유전자만 사용하고 싶다면 genes 인자에 리스트 전달 가능
# 여기서는 모든 공통 유전자를 사용하도록 함
# tg.pp_adatas(ad_sc, ad_sp, genes=None) # genes=None이면 공통 유전자 사용

# # 세포 매핑
# # device 설정: GPU 사용 가능 시 'cuda:0', 아니면 'cpu'
# # import torch
# # device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# device = 'cpu' # CPU 사용 예시
# ad_map = tg.map_cells_to_space(ad_sc, ad_sp, device=device)

# # 유전자 발현 예측 (Imputation/Projection)
# # mode='clusters' 사용 시 cell type annotation 필요 (예: ad_sc.obs['cell_type'])
# # 여기서는 개별 세포 기반 예측 ('cells') 사용 가정
# # project_genes는 ad_sp의 공간 구조를 기반으로 ad_ge를 생성
# ad_ge = tg.project_genes(ad_map, ad_sc, cluster_label=None) # cluster_label=None for 'cells' mode

# ad_ge의 관측치(obs)는 ad_sp의 관측치(세포)와 같음. 공간 좌표도 유지됨.
# 하지만 project_genes의 결과는 각 공간 위치(여기선 세포 위치)에서의 예측값이 됨.
# 만약 입력 ad_sp가 이미 spot 레벨이었다면 ad_ge도 spot 레벨이 됨.
# Xenium의 경우 ad_sp가 cell 레벨이므로, ad_ge도 cell 레벨의 예측값을 가짐.
# -> 이 경우, 사용자의 원래 문제의식(voxel vs cell)과는 약간 상황이 다름.
#    만약 사용자의 ad_sp가 Visium처럼 spot 레벨이었다면 다음 aggregation이 필수.
#    Xenium 데이터라도, 분석 목적상 spot/voxel 레벨로 결과를 보고 싶다면 aggregation 수행.

# 여기서는 ad_ge가 'spot' 레벨이라고 가정하고 Aggregation을 진행.
# 이를 위해 ad_ge를 가상으로 spot 레벨 데이터로 변환하는 단계를 추가함.
# --- 가상 Spot 레벨 ad_ge 생성 (실제로는 project_genes 결과를 사용) ---
# 실제로는 project_genes의 결과가 어떤 형태인지(cell or spot 레벨) 확인 필요
# 여기서는 설명을 위해 ad_ge를 spot 레벨 데이터로 재구성
spot_coords = rng.random((n_spots, 2)) * 100
ad_ge_X = rng.poisson(1, size=(n_spots, n_genes)).astype(np.float32)
ad_ge_obs = pd.DataFrame(index=[f"Spot_{i}" for i in range(n_spots)])
ad_ge_var = ad_sc.var.copy() # 유전자 정보는 ad_sc에서 가져옴
# 공간 좌표를 .obsm에 저장
ad_ge = ad.AnnData(X=ad_ge_X, obs=ad_ge_obs, var=ad_ge_var)
ad_ge.obsm['spatial'] = spot_coords

print("Tangram projected data (ad_ge, assumed spot-level):", ad_ge)
print("-" * 30)


# --- 4. 원본 공간 데이터 집계 (`ad_sp` -> `ad_sp_agg`) ---
print("Aggregating original spatial data (ad_sp) to match ad_ge structure...")

# ad_ge의 스팟 좌표 (기준)
coords_ge = ad_ge.obsm['spatial']
# ad_sp의 세포 좌표
coords_sp = ad_sp.obsm['spatial']

# KDTree를 사용하여 각 세포(ad_sp)에 가장 가까운 스팟(ad_ge) 찾기
kdtree = KDTree(coords_ge)
# query 결과: distances (거리), indices (가장 가까운 이웃의 인덱스 - ad_ge 기준)
distances, indices = kdtree.query(coords_sp)

# 각 세포가 어떤 스팟에 할당되었는지 기록 (ad_ge의 obs_names 사용)
ad_sp.obs['spot_assignment'] = ad_ge.obs_names[indices]

# 스팟별로 평균 유전자 발현 계산
# 결과를 저장할 DataFrame 생성 (인덱스: 스팟 이름, 컬럼: 유전자 이름)
agg_expression_df = pd.DataFrame(np.nan, index=ad_ge.obs_names, columns=ad_sp.var_names)

# 각 스팟에 대해
for spot_name in ad_ge.obs_names:
    # 해당 스팟에 할당된 세포들의 인덱스를 찾음
    cells_in_spot = ad_sp.obs_names[ad_sp.obs['spot_assignment'] == spot_name]

    if len(cells_in_spot) > 0:
        # 해당 세포들의 발현 데이터 추출 (AnnData 슬라이싱)
        # .X는 sparse일 수 있으므로 .toarray() 사용 고려 (메모리 주의)
        # 여기서는 mean 계산 시 자동으로 dense 변환될 수 있음
        spot_mean_expression = ad_sp[cells_in_spot, :].X.mean(axis=0)
        # 결과 저장 (sparse matrix의 mean 결과는 1D numpy array일 수 있음)
        if isinstance(spot_mean_expression, np.matrix):
            agg_expression_df.loc[spot_name] = spot_mean_expression.A1 # matrix를 1D array로
        else:
             agg_expression_df.loc[spot_name] = spot_mean_expression

# 집계된 데이터를 AnnData 객체로 생성
# NaN 값 처리: 스팟에 할당된 세포가 없는 경우 NaN이 됨. 0으로 채우거나 그대로 둠.
# 여기서는 0으로 채우는 것을 선택 (분석 목적에 따라 다를 수 있음)
agg_expression_filled = agg_expression_df.fillna(0).values

ad_sp_agg = ad.AnnData(X=agg_expression_filled,
                       obs=ad_ge.obs.copy(), # obs는 ad_ge와 동일하게 설정
                       var=ad_sp.var.copy()) # var는 원본 ad_sp와 동일하게 설정
# 공간 좌표도 ad_ge에서 복사
ad_sp_agg.obsm['spatial'] = ad_ge.obsm['spatial']

print("Aggregated spatial data (ad_sp_agg):", ad_sp_agg)
print("-" * 30)

# --- 5. 데이터 정렬 및 필터링 ---
print(f"Filtering data for {len(evaluation_genes)} evaluation genes...")

# 공통 유전자 확인 (이미 Tangram 단계에서 처리되었을 수 있지만, 집계 과정 등에서 달라질 수 있으므로 재확인)
common_genes = list(set(evaluation_genes).intersection(ad_ge.var_names).intersection(ad_sp_agg.var_names))
print(f"Found {len(common_genes)} common genes for evaluation.")

if not common_genes:
    raise ValueError("No common genes found between ad_ge and ad_sp_agg for evaluation.")

# 평가 대상 유전자 및 스팟으로 데이터 필터링
# 스팟 순서는 이미 ad_sp_agg 생성 시 ad_ge와 동일하게 맞춤
ad_ge_eval = ad_ge[:, common_genes].copy()
ad_sp_agg_eval = ad_sp_agg[:, common_genes].copy()

print("Filtered ad_ge for evaluation:", ad_ge_eval)
print("Filtered ad_sp_agg for evaluation:", ad_sp_agg_eval)
print("-" * 30)

# --- 6. 평가 지표 계산 ---
print("Calculating evaluation metrics...")

# 결과를 저장할 딕셔너리
results = {
    'gene_spearman': {},
    'gene_pearson': {},
    'spot_spearman': {},
    'spot_pearson': {},
    'overall_spearman': None,
    'overall_pearson': None,
    'rmse': None,
    'mae': None
}

# 계산에 사용할 데이터 추출 (메모리를 고려하여 필요 시 sparse 유지, 여기서는 dense 변환)
X_ge = ad_ge_eval.X if isinstance(ad_ge_eval.X, np.ndarray) else ad_ge_eval.X.toarray()
X_sp_agg = ad_sp_agg_eval.X if isinstance(ad_sp_agg_eval.X, np.ndarray) else ad_sp_agg_eval.X.toarray()

# 유전자별 상관계수
for gene in common_genes:
    vec_ge = X_ge[:, ad_ge_eval.var_names.get_loc(gene)]
    vec_sp_agg = X_sp_agg[:, ad_sp_agg_eval.var_names.get_loc(gene)]

    # 유효한 값 (NaN이 아닌)이 2개 이상 있어야 상관계수 계산 가능
    valid_mask = ~np.isnan(vec_ge) & ~np.isnan(vec_sp_agg)
    if np.sum(valid_mask) >= 2:
        spearman_corr, _ = spearmanr(vec_ge[valid_mask], vec_sp_agg[valid_mask])
        pearson_corr, _ = pearsonr(vec_ge[valid_mask], vec_sp_agg[valid_mask])
        results['gene_spearman'][gene] = spearman_corr if not np.isnan(spearman_corr) else 0
        results['gene_pearson'][gene] = pearson_corr if not np.isnan(pearson_corr) else 0
    else:
        results['gene_spearman'][gene] = 0
        results['gene_pearson'][gene] = 0


# 스팟별 상관계수
for i, spot_name in enumerate(ad_ge_eval.obs_names):
    vec_ge = X_ge[i, :]
    vec_sp_agg = X_sp_agg[i, :]

    valid_mask = ~np.isnan(vec_ge) & ~np.isnan(vec_sp_agg)
    if np.sum(valid_mask) >= 2:
        spearman_corr, _ = spearmanr(vec_ge[valid_mask], vec_sp_agg[valid_mask])
        pearson_corr, _ = pearsonr(vec_ge[valid_mask], vec_sp_agg[valid_mask])
        results['spot_spearman'][spot_name] = spearman_corr if not np.isnan(spearman_corr) else 0
        results['spot_pearson'][spot_name] = pearson_corr if not np.isnan(pearson_corr) else 0
    else:
        results['spot_spearman'][spot_name] = 0
        results['spot_pearson'][spot_name] = 0


# 전체 데이터에 대한 상관계수 및 오차
valid_mask_flat = ~np.isnan(X_ge.flatten()) & ~np.isnan(X_sp_agg.flatten())
if np.sum(valid_mask_flat) >= 2:
    overall_spearman, _ = spearmanr(X_ge.flatten()[valid_mask_flat], X_sp_agg.flatten()[valid_mask_flat])
    overall_pearson, _ = pearsonr(X_ge.flatten()[valid_mask_flat], X_sp_agg.flatten()[valid_mask_flat])
    results['overall_spearman'] = overall_spearman if not np.isnan(overall_spearman) else 0
    results['overall_pearson'] = overall_pearson if not np.isnan(overall_pearson) else 0
else:
    results['overall_spearman'] = 0
    results['overall_pearson'] = 0

# RMSE (Root Mean Squared Error)
results['rmse'] = np.sqrt(np.nanmean((X_ge - X_sp_agg)**2))
# MAE (Mean Absolute Error)
results['mae'] = np.nanmean(np.abs(X_ge - X_sp_agg))

print("Evaluation metrics calculated.")
print(f"  Average gene Spearman corr: {np.mean(list(results['gene_spearman'].values())):.4f}")
print(f"  Average gene Pearson corr: {np.mean(list(results['gene_pearson'].values())):.4f}")
print(f"  Average spot Spearman corr: {np.mean(list(results['spot_spearman'].values())):.4f}")
print(f"  Average spot Pearson corr: {np.mean(list(results['spot_pearson'].values())):.4f}")
print(f"  Overall Spearman corr: {results['overall_spearman']:.4f}")
print(f"  Overall Pearson corr: {results['overall_pearson']:.4f}")
print(f"  RMSE: {results['rmse']:.4f}")
print(f"  MAE: {results['mae']:.4f}")
print("-" * 30)

# --- 7. 결과 시각화 (예시) ---
print("Visualizing results...")

# 결과 DataFrame 생성 (유전자별 상관계수)
gene_corr_df = pd.DataFrame({
    'Spearman': results['gene_spearman'],
    'Pearson': results['gene_pearson']
})

# 유전자별 상관계수 히스토그램
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(gene_corr_df['Spearman'], kde=True)
plt.title('Distribution of Gene-wise Spearman Correlation')
plt.xlabel('Spearman Correlation')

plt.subplot(1, 2, 2)
sns.histplot(gene_corr_df['Pearson'], kde=True, color='salmon')
plt.title('Distribution of Gene-wise Pearson Correlation')
plt.xlabel('Pearson Correlation')

plt.tight_layout()
plt.show()

# 특정 유전자에 대한 발현량 비교 산점도
example_gene = common_genes[0] # 첫 번째 유전자 예시
vec_ge = X_ge[:, ad_ge_eval.var_names.get_loc(example_gene)]
vec_sp_agg = X_sp_agg[:, ad_sp_agg_eval.var_names.get_loc(example_gene)]

plt.figure(figsize=(6, 6))
sns.scatterplot(x=vec_sp_agg, y=vec_ge)
plt.xlabel(f"Aggregated Original Expression ({example_gene})")
plt.ylabel(f"Tangram Imputed Expression ({example_gene})")
plt.title(f"Expression Comparison for {example_gene}")
# 추세선 추가 (선택 사항)
sns.regplot(x=vec_sp_agg, y=vec_ge, scatter=False, color='red', line_kws={'linewidth':1})
# 상관계수 표시
corr_text = f"Spearman: {results['gene_spearman'][example_gene]:.3f}\nPearson: {results['gene_pearson'][example_gene]:.3f}"
plt.text(0.05, 0.95, corr_text, transform=plt.gca().transAxes, ha='left', va='top', bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.5))
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

# 공간적 시각화 (선택 사항 - Scanpy 사용)
# ad_ge_eval과 ad_sp_agg_eval에 대해 특정 유전자 발현 플롯
# sc.pl.spatial(ad_sp_agg_eval, color=[example_gene], title=[f"Aggregated Original ({example_gene})"])
# sc.pl.spatial(ad_ge_eval, color=[example_gene], title=[f"Tangram Imputed ({example_gene})"])

print("Evaluation process finished.")